## Container Terminal Congestion Problem

Consider a container terminal with a storage yard consisting of 100 blocks, each with
storage space to hold 600 containers, numbered 1 to 100. 

For $i = 1, \dots, 100$, if no newly arriving containers are sent to block $i$, the number 
of stored containers is expected to be:

320, 157, 213, 96, 413, 312, 333, 472, 171, 222,
439, 212, 190, 220, 372, 101, 212, 251, 86, 79,
295, 138, 343, 281, 372, 450, 100, 183, 99, 505,
99, 254, 330, 279, 300, 150, 340, 221, 79, 119,
43, 71, 219, 363, 98, 500, 413, 259, 182, 391,
360, 447, 181, 233, 414, 30, 333, 427, 251, 83,
144, 404, 76, 84, 196, 336, 411, 280, 115, 200,
117, 284, 263, 477, 431, 297, 380, 327, 155, 360,
360, 290, 350, 157, 116, 141, 82, 116, 99, 78,
220, 182, 96, 301, 121, 278, 372, 119, 282, 310


The terminal estimates that **15,166 new containers** will be unloaded and dispatched 
to the storage yard in this period.  

**Goal:** Determine an optimal plan to allocate these new containers to blocks to 
**minimize congestion** on the terminal roads.

---

## Import the libraries

In [22]:
import pyomo
import numpy as np
import pandas as pd

from pyomo.environ import SolverFactory
import pyomo.environ as pyo
from pyomo.environ import RangeSet, NonNegativeReals, Reals, Constraint

## Model Inputs

In [13]:
# a_i: The number of stored containers in block i at the end of the period if no new containers are sent

a_i = [320, 157, 213, 96, 413, 312, 333, 472, 171, 222,
439, 212, 190, 220, 372, 101, 212, 251, 86, 79,
295, 138, 343, 281, 372, 450, 100, 183, 99, 505,
99, 254, 330, 279, 300, 150, 340, 221, 79, 119,
43, 71, 219, 363, 98, 500, 413, 259, 182, 391,
360, 447, 181, 233, 414, 30, 333, 427, 251, 83,
144, 404, 76, 84, 196, 336, 411, 280, 115, 200,
117, 284, 263, 477, 431, 297, 380, 327, 155, 360,
360, 290, 350, 157, 116, 141, 82, 116, 99, 78,
220, 182, 96, 301, 121, 278, 372, 119, 282, 310]

a_i = {i: value_i for i, value_i in enumerate(a_i, 1)}

# B: The number of blocks in storage yard
# A: The number of storage space in each block

B = 100
A = 600

period = 4  # 4 hours

# N: The number of new containers being unloaded and dispatched to the storage yard
N = 15166


# Calculate Fill-Ration
F = (N + sum(a_i))/A*B

## LP with Pyomo

In [6]:
solver = SolverFactory('glpk')

In [ ]:
model = pyo.ConcreteModel()

model.blocks = RangeSet(1, B)

model.x = pyo.Var(model.blocks, within=NonNegativeReals)
model.u_plus = pyo.Var(model.blocks, within=Reals)
model.u_minus = pyo.Var(model.blocks, within=Reals)

def congestion_rule(model, i):
    return a_i[i] * model.x[i] - (model.u_plus[i] - model.u_minus[i]) == A * F

model.congestion_constraint = Constraint(model.blocks, rule=congestion_rule)
